In [1]:
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Part - I

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).content
soup = BeautifulSoup(result, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

cols = ['PostalCode', 'Borough', 'Neighborhood']
df=pd.DataFrame(table_contents, columns=cols)
print(df.shape)

(103, 3)


In [3]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Part - II

In [4]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()
df.shape

(103, 3)

### Coordinates of the Neighborhood

In [5]:
#cant get geocoder.google to work so using csv file
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/
df_latlong = pd.read_csv('Geospatial_Coordinates.csv') # Read the csv data
df_latlong.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# Create new columns in df
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df_cord, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df.index:
    c_idx = df_latlong['Postal Code'] == df.loc[idx, 'PostalCode']
    df.at[idx, 'Latitude'] = df_latlong.loc[c_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_latlong.loc[c_idx, 'Longitude'].values

In [7]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part - III

In [8]:
df.loc[df['PostalCode'] == 'M4T']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316


## Toronto Map

In [9]:
geoloc = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geoloc.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

Tmap = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Tmap)  
    
Tmap

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


## Using Foursquare to Analyze location

In [10]:
CLIENT_ID = '2ZN3TPSTZ2YHCDFULTDGFVGBX1YJL45HQ0LD3RCFOEMXCMM5' # ID
CLIENT_SECRET = '3KYZWRDZC3JJNDBOS5V1MPTYUZPMGKNRI0CMKLHR125T0NHW' # Secret
VERSION = '20210508'

In [11]:
CT_latitude = df.loc[48]['Latitude']
CT_longitude = df.loc[48]['Longitude']

In [12]:
CT_latitude

43.6895743

In [13]:
CT_longitude

-79.38315990000001

In [14]:
LIMIT = 100 # limit to number of venues
radius = 600 # defining scope
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CT_latitude, 
    CT_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2ZN3TPSTZ2YHCDFULTDGFVGBX1YJL45HQ0LD3RCFOEMXCMM5&client_secret=3KYZWRDZC3JJNDBOS5V1MPTYUZPMGKNRI0CMKLHR125T0NHW&v=20210508&ll=43.6895743,-79.38315990000001&radius=600&limit=100'

In [15]:
result = requests.get(url).json()

In [16]:
# function that will return the category of the venue
def get_type(row):
    try:
        categories = row['categories']
    except:
        categories = row['venue.categories']
        
    if len(categories) == 0:
        return None
    else:
        return categories[0]['name']

In [17]:
venues = result['response']['groups'][0]['items']

### Normalize JSON

In [18]:
nearby_venues = json_normalize(venues)
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bc0d290920eb713dda1192c-0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4bc0d290920eb713dda1192c,75 Rosehill Avenue,CA,Toronto,Canada,NaN,541,"[75 Rosehill Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.68630352898223...",43.686304,-79.388133,NaN,ON,Rosehill Reservoir,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bf031dcf831c928eab401f2-1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4bf031dcf831c928eab401f2,200 Mount Pleasant Rd,CA,Toronto,Canada,NaN,591,"[200 Mount Pleasant Rd, Toronto ON M4T 2C4, Ca...","[{'label': 'display', 'lat': 43.68562946634291...",43.685629,-79.388078,M4T 2C4,ON,David A. Balfour Park,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bb0200cf964a52090383ce3-2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",4bb0200cf964a52090383ce3,939 Eglinton Ave E,CA,Toronto,Canada,Brentcliffe,533,"[939 Eglinton Ave E (Brentcliffe), Toronto ON ...","[{'label': 'display', 'lat': 43.69300659993013...",43.693007,-79.387781,M4G 4E8,ON,Pure Fitness,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4db7452da86ed8d46c7117f9-3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4db7452da86ed8d46c7117f9,276 St. Clair Avenue East,CA,Toronto,Canada,NaN,80,"[276 St. Clair Avenue East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.69027042721738...",43.690270,-79.383438,NaN,ON,Loring-Wyle Parkette,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bb7c99253649c741a4746fb-4,"[{'id': '4e39a956bd410d7aed40cbc3', 'name': 'T...",4bb7c99253649c741a4746fb,175 Moore Ave,CA,Toronto,Canada,NaN,414,"[175 Moore Ave, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.69328930873421...",43.693289,-79.382900,NaN,ON,Moore Park Tennis Club,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ae73286f964a5204aa921e3-5,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",4ae73286f964a5204aa921e3,Moore Ave.,CA,Toronto,Canada,at Welland Ave.,449,"[Moore Ave. (at Welland Ave.), Toronto ON, Can...","[{'label': 'display', 'lat': 43.69360987933692...",43.693610,-79.383465,NaN,ON,Moorevale Park,0,[]


In [19]:
# firstly filtering columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# Moving on to the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_type, axis=1)

# cleaning columns for better representation
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosehill Reservoir,Park,43.686304,-79.388133
1,David A. Balfour Park,Park,43.685629,-79.388078
2,Pure Fitness,Gym,43.693007,-79.387781
3,Loring-Wyle Parkette,Park,43.690270,-79.383438
4,Moore Park Tennis Club,Tennis Court,43.693289,-79.382900


In [20]:
print('Foursquare returned {} venues for Central Toronto'.format(nearby_venues.shape[0]))

Foursquare returned 6 venues for Central Toronto


In [21]:
def NearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # getting url in result
        result = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in result])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
venues = NearbyVenues(names=df['Borough'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East YorkEast Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown TorontoStn A PO Boxes25 The Esplanade
Downtown Toronto
North York

In [23]:
print(venues.shape)
venues.head()

(2126, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Number of Venues per Neighborhood

In [24]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1099,1099,1099,1099,1099,1099
Downtown TorontoStn A PO Boxes25 The Esplanade,100,100,100,100,100,100
East Toronto,103,103,103,103,103,103
East TorontoBusiness reply mail Processing Centre969 Eastern,17,17,17,17,17,17
East York,70,70,70,70,70,70
East YorkEast Toronto,4,4,4,4,4,4
Etobicoke,67,67,67,67,67,67
EtobicokeNorthwest,3,3,3,3,3,3


In [25]:
print('There are {} unique venue categories in Toronto.'.format(len(venues['Venue Category'].unique())))

There are 268 unique venue categories in Toronto.


In [26]:
dummy = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dummy['Neighborhood'] = venues['Neighborhood'] 

dummy.head()
print(dummy.shape)
groups = dummy.groupby('Neighborhood').mean().reset_index()
groups

(2126, 268)


,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.008929,...,0.00000,0.000000,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000000,0.00091,0.000910,0.00091,0.00091,0.00182,0.00273,0.00091,0.011829,...,0.00273,0.010919,0.000910,0.003640,0.000000,0.007279,0.000910,0.000000,0.000000,0.004550
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
3,East Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.029126,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
4,East TorontoBusiness reply mail Processing Cen...,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,East York,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,0.000000,0.000000,0.014286
6,East YorkEast Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Etobicoke,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.014925,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000
8,EtobicokeNorthwest,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,MississaugaCanada Post Gateway Processing Centre,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.071429,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
num_top_venues = 5

for g in groups['Neighborhood']:
    print("----"+g+"----")
    temp = groups[groups['Neighborhood'] == g].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.07
2            Park  0.06
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2           Restaurant  0.03
3                Hotel  0.03
4  Japanese Restaurant  0.03


----Downtown TorontoStn A PO Boxes25 The Esplanade----
                venue  freq
0         Coffee Shop  0.12
1  Seafood Restaurant  0.04
2        Cocktail Bar  0.04
3  Italian Restaurant  0.03
4            Beer Bar  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4             Brewery  0.04


----East TorontoBusiness reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1                Park  0.06
2       Burrito Place  0.06
3       Garden Center  0.06
4   

In [28]:
def common_venues(row, num_venues):
    categories = row.iloc[1:]
    categories_sorted = categories.sort_values(ascending=False)
    
    return categories_sorted.index.values[0:num_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create new columns according to the number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = groups['Neighborhood']

for ind in np.arange(groups.shape[0]):
    venues_sorted.iloc[ind, 1:] = common_venues(groups.iloc[ind, :], num_top_venues)

venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Clothing Store,Dessert Shop,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Clothing Store,Seafood Restaurant
2,Downtown TorontoStn A PO Boxes25 The Esplanade,Coffee Shop,Seafood Restaurant,Cocktail Bar,Restaurant,Café,Beer Bar,Hotel,Bakery,Italian Restaurant,Japanese Restaurant
3,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Park,Pub,Bakery,American Restaurant,Café
4,East TorontoBusiness reply mail Processing Cen...,Light Rail Station,Recording Studio,Spa,Restaurant,Butcher,Skate Park,Burrito Place,Fast Food Restaurant,Farmers Market,Garden
5,East York,Coffee Shop,Bank,Burger Joint,Pizza Place,Liquor Store,Sporting Goods Shop,Indian Restaurant,Beer Store,Skating Rink,Supermarket
6,East YorkEast Toronto,Park,Convenience Store,Coffee Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Bakery,Café,Grocery Store,Gym,Park
8,EtobicokeNorthwest,Drugstore,Garden Center,Rental Car Location,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
9,MississaugaCanada Post Gateway Processing Centre,Coffee Shop,Hotel,Burrito Place,Sandwich Place,Gas Station,Middle Eastern Restaurant,Intersection,American Restaurant,Fried Chicken Joint,Mediterranean Restaurant


In [30]:
# set number of clusters
clusters = 5

clustering = groups.drop('Neighborhood', 1)

# k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(clustering)

# check cluster labels in the dataframe generated for each row
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 4, 0, 2, 3])

In [31]:
# set number of clusters
clusters = 5

clustering = groups.drop('Neighborhood', 1)

# k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(clustering)

# check cluster labels in the dataframe generated for each row
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 4, 0, 2, 3])

In [32]:
merged = groups

# adding cluster labels
merged['Cluster Labels'] = kmeans.labels_

# merging data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.008929,...,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Clothing Store,Dessert Shop,Gym
1,Downtown Toronto,0.0,0.00091,0.00091,0.00091,0.00091,0.00182,0.00273,0.00091,0.011829,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Clothing Store,Seafood Restaurant
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,Coffee Shop,Seafood Restaurant,Cocktail Bar,Restaurant,Café,Beer Bar,Hotel,Bakery,Italian Restaurant,Japanese Restaurant
3,East Toronto,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.029126,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Park,Pub,Bakery,American Restaurant,Café
4,East TorontoBusiness reply mail Processing Cen...,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,...,Light Rail Station,Recording Studio,Spa,Restaurant,Butcher,Skate Park,Burrito Place,Fast Food Restaurant,Farmers Market,Garden
